In [4]:
import cirq
import sympy

In [5]:
# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit
circuit = cirq.Circuit(
    cirq.X(qubit)**0.5,  # Square root of NOT.
    cirq.measure(qubit, key='m')  # Measurement.
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)


Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=00010000110111111001


In [6]:
# qubits = cirq.LineQubit.range(5)

# circuit = cirq.Circuit()

# for q in qubits:
#     circuit.append(cirq.H(q))

# circuit.append()



# print(circuit)

In [16]:
import pennylane as qml
import pennylane.numpy as np
import tensorflow as tf

In [8]:
from pennylane import qchem
from functools import partial

In [9]:
def info_from_data(data):
    symbols = []
    coordinates = []
    for par in data:
        atom, coordinate = par[0], par[1]
        symbols.append(atom)
        for x in coordinate:
            coordinates.append(x)
    return (symbols, np.array(coordinates))

# symbols = ["H", "H"]
# coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

data = [["H", [0.0, 0.0, -0.6614]], ["H", [0.0, 0.0, 0.6614]]]
# data = [["Li", [0, 0, 0]], ["H", [1, 0, 0]]]
symbols, coordinates = info_from_data(data)
print(symbols)
print(coordinates)

['H', 'H']
[ 0.      0.     -0.6614  0.      0.      0.6614]


In [10]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print(f"The Hamiltonian is {H}")

Number of qubits =  4
The Hamiltonian is   (-0.2427450125037262) [Z3]
+ (-0.24274501250372615) [Z2]
+ (-0.0420725520411073) [I0]
+ (0.17771358235554002) [Z0]
+ (0.1777135823555401) [Z1]
+ (0.12293330446041997) [Z0 Z2]
+ (0.12293330446041997) [Z1 Z3]
+ (0.16768338851158246) [Z0 Z3]
+ (0.16768338851158246) [Z1 Z2]
+ (0.1705975927541784) [Z0 Z1]
+ (0.1762766138630638) [Z2 Z3]
+ (-0.04475008405116248) [Y0 Y1 X2 X3]
+ (-0.04475008405116248) [X0 X1 Y2 Y3]
+ (0.04475008405116248) [Y0 X1 X2 Y3]
+ (0.04475008405116248) [X0 Y1 Y2 X3]


In [11]:
electrons = 2

hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 0 0]


In [12]:
# Generate single and double excitations
singles, doubles = qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)

In [14]:
# use cirq simulator
dev = qml.device('cirq.simulator', wires=qubits)

# Define the UCCSD ansatz
ansatz = partial(qml.UCCSD, init_state=hf, s_wires=s_wires, d_wires=d_wires)

# Define the cost function
cost_fn = qml.ExpvalCost(ansatz, H, dev)

In [17]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# theta = np.array(0.0, requires_grad=True)
theta = np.random.normal(0, np.pi, len(singles)+len(doubles), requires_grad=True)


In [18]:
energy = [cost_fn(theta)]

angle = [theta]

max_iterations = 50
conv_tol = 1e-6

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 5 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")

print(f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print(f"Optimal value of the circuit parameter = {angle[-1]}")



Step = 0, Energy = -0.70846057 Ha
Step = 5, Energy = -1.08742478 Ha
Step = 10, Energy = -1.13202248 Ha


KeyboardInterrupt: 